In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('dns_logs.txt.gz', compression='gzip', sep="\t", header=None)
df

,0,1,2
0,1.677974e+09,128.61.96.194,tiktok.investing.com
1,1.677974e+09,130.207.32.200,s-img.ukr.net
2,1.677974e+09,130.207.88.43,www.lichnosti.net
3,1.677974e+09,128.61.78.96,google-bidout-d.vgsgaming-ads.com
4,1.677974e+09,128.61.225.148,static.seoclerk.com
...,...,...,...
9534819,1.678579e+09,143.215.119.63,129-137-96-17_s-23-55-220-75_ts-1696895543-cli...
9534820,1.678579e+09,128.61.83.16,static.signsonthecheap.com
9534821,1.678579e+09,143.215.213.140,collect.prnx.net
9534822,1.678579e+09,130.207.213.18,not4u.mechaworld.ru


In [8]:
# Get the frequency of unique requests by each user to each domain
sizes = df.groupby([df[1], df[2]],as_index=False).size()

In [10]:
sizes.sort_values(by='size', ascending=False)

,1,2,size
9054846,143.215.58.72,c2.robotland.org,2017
5861665,130.207.47.207,c2.robotland.org,2016
4828480,130.207.213.18,not4u.mechaworld.ru,1009
1064510,128.61.188.8,not4u.mechaworld.ru,1009
6520614,143.215.0.38,not4u.mechaworld.ru,1008
...,...,...,...
3175412,130.207.114.135,edge.youtubetob.com,1
3175413,130.207.114.135,encrypted-tbn3.leguide.com,1
3175414,130.207.114.135,external-preview.multitran.ru,1
3175415,130.207.114.135,googleads.about.com,1


In [16]:
sizes[sizes['size']>2]

,1,2,size
1064510,128.61.188.8,not4u.mechaworld.ru,1009
1570886,128.61.230.93,yhexyweqm.picklepalace.mx,312
2187098,128.61.54.78,www.tanglerootinn.in.ua,336
3082992,130.207.106.45,www.tanglerootinn.in.ua,337
4828480,130.207.213.18,not4u.mechaworld.ru,1009
5175746,130.207.243.14,hamma.doubutsutaikai.jp,318
5861665,130.207.47.207,c2.robotland.org,2016
6520614,143.215.0.38,not4u.mechaworld.ru,1008
7499578,143.215.182.109,mx.yellowjello.br,80
8656626,143.215.26.32,mx.yellowjello.br,80


In [27]:
# All of the domains with more than 2 requests from the same IP are always accessed many times, never just once or twice. 
# Show this:
problems = sizes[sizes['size']>2]
for dom in problems[2].unique().tolist():
    print('domain:', dom, '| unique IPs:', len(sizes[sizes[2]==dom]), '| min. requests:', min(sizes[sizes[2]==dom]['size']))

domain: not4u.mechaworld.ru | unique IPs: 3 | min. requests: 1008
domain: yhexyweqm.picklepalace.mx | unique IPs: 1 | min. requests: 312
domain: www.tanglerootinn.in.ua | unique IPs: 2 | min. requests: 336
domain: hamma.doubutsutaikai.jp | unique IPs: 1 | min. requests: 318
domain: c2.robotland.org | unique IPs: 2 | min. requests: 2016
domain: mx.yellowjello.br | unique IPs: 2 | min. requests: 80


In [28]:
# Sure enough, these domains are only accessed by a select few IPs and always have many requests. 
# Prepare the answers:
import ipaddress
import hashlib

iplist = problems[1].unique().tolist()
domlist = problems[2].unique().tolist()

ipsorted = [str(x) for x in sorted(ipaddress.ip_address(ip) for ip in iplist)]
domsorted = sorted(domlist)

hashinput = ", ".join(ipsorted + domsorted)
answer_hash = hashlib.md5(hashinput.encode('utf-8')).hexdigest()
print(answer_hash) # => 'd209f629770e87c8c5834f890022fd44'

2221e46d1063a69dfe861c620d2b2851
